In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
from matplotlib import pyplot as plt
import sys
import matplotlib.animation as animation
import scipy.sparse as sparse
import scipy.io as sio
import scipy.stats as stats
import scipy.spatial.distance as dist
import numpy as np
import h5py
from tqdm import tqdm
import pandas as pd
import imageio
import os 

#import tables
#import deepdish as dd

sys.path.append('/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis')
import python_utils_jjm as jjm_utils

In [3]:
dir_files = [f_name for f_name in os.listdir('/Volumes/My_Passport/cnmfe_analysis_files/batch_output_files/') if 'out.mat' in f_name]

In [4]:
results = sio.loadmat('/Volumes/My_Passport/cnmfe_analysis_files/batch_output_files/' + '08-Apr_16_03_01_out.mat')

In [5]:
#results['P'][0][0][13]

In [6]:
## return a dictionary matching the batch output file to the information from quest directories 
file_info = {}
files_with_errors = []
for file in dir_files:
    results = sio.loadmat('/Volumes/My_Passport/cnmfe_analysis_files/batch_output_files/' + file)
    try:
        file_ids = [results['P'][0][0][13][0][0][0].split('/')[y] for y in range(5, 7)]
        file_ids.append([results['P'][0][0][13][batch_folder][0][0].split('/')[7] for batch_folder in range(len(results['P'][0][0][13]))])
        file_ids.append([results['P'][0][0][13][batch_folder][0][0].split('/')[8] for batch_folder in range(len(results['P'][0][0][13]))])
        file_info[file] = file_ids
        print(file)
        print('ok')
    except IndexError:
        print('error with')
        print(file)
        files_with_errors.append(file)
        pass
    

08-Apr_16_03_01_out.mat
ok
08-Apr_22_54_12_out.mat
ok
10-Jan_20_51_00_out.mat
ok
10-Jan_21_07_47_out.mat
ok
11-Jan_11_46_12_out.mat
ok
11-Jan_12_56_26_out.mat
ok
12-Apr_19_48_06_out.mat
ok
12-Apr_21_20_00_out.mat
ok
12-Mar_21_45_41_out.mat
ok
error with
13-Apr_15_30_16_out.mat
13-Apr_16_01_20_out.mat
ok
13-Apr_16_11_27_out.mat
ok
13-Apr_17_57_40_out.mat
ok
22-Mar_22_52_02_out.mat
ok
22-Mar_22_52_02_test_out.mat
ok
25-Mar_13_27_27_out.mat
ok
25-Mar_14_22_02_out.mat
ok
25-Mar_14_22_44_out.mat
ok
26-Mar_18_33_55_out.mat
ok
error with
26-Mar_22_01_01_out.mat
27-Feb_17_32_15_out.mat
ok
27-Feb_17_33_59_out.mat
ok
27-Mar_00_26_12_out.mat
ok
27-Mar_00_48_46_out.mat
ok
28-Feb_16_10_05_out.mat
ok
28-Feb_16_21_21_out.mat
ok
error with
29-Mar_11_29_08_out.mat
29-Mar_13_39_44_out.mat
ok
29-Mar_14_27_55_out.mat
ok
error with
29-Mar_20_07_14_out.mat
29-Mar_21_42_20_out.mat
ok
error with
30-Mar_11_06_01_out.mat
30-Mar_20_39_05_out.mat
ok
30-Mar_20_45_16_out.mat
ok
31-Jan_14_07_30_out.mat
ok
error with

In [7]:
file_info['08-Apr_16_03_01_out.mat'][0]

'GRIN035'

In [19]:
file_info['31-Mar_13_28_15_out.mat'][0]

'H13_M59_S14'

In [7]:
# display list of file with associated animal numbers
mouse_file_dict = {}
for cnmfe_file in list(file_info.keys()):
    mouse_file_dict[cnmfe_file] = [file_info[cnmfe_file][0], file_info[cnmfe_file][1]]
    

In [8]:
mouse_file_dict

{'08-Apr_16_03_01_out.mat': ['GRIN035', 'rotarodrecordings'],
 '08-Apr_22_54_12_out.mat': ['GRIN035', 'rotarodrecordings'],
 '10-Jan_20_51_00_out.mat': ['GRIN032', 'H17_M30_S22'],
 '10-Jan_21_07_47_out.mat': ['GRIN026', 'H16_M35_S34'],
 '11-Jan_11_46_12_out.mat': ['GRIN032', 'H17_M30_S22'],
 '11-Jan_12_56_26_out.mat': ['GRIN032', 'H17_M30_S22'],
 '12-Apr_19_48_06_out.mat': ['GRIN035', 'rotarod'],
 '12-Apr_21_20_00_out.mat': ['GRIN035', 'rotarod'],
 '12-Mar_21_45_41_out.mat': ['memmap_0000memmap_0004_resized_source_extraction_03_12_20_T_19_58',
  'frames_1_5000'],
 '13-Apr_16_01_20_out.mat': ['GRIN012', 'H17_M46_S35'],
 '13-Apr_16_11_27_out.mat': ['GRIN012', 'H17_M32_S17'],
 '13-Apr_17_57_40_out.mat': ['GRIN018', 'H16_M13_S53'],
 '22-Mar_22_52_02_out.mat': ['GRIN033', 'H14_M34_S32'],
 '22-Mar_22_52_02_test_out.mat': ['GRIN033', 'H14_M34_S32'],
 '25-Mar_13_27_27_out.mat': ['GRIN034', 'H15_M28_S2'],
 '25-Mar_14_22_02_out.mat': ['GRIN034', 'H16_M13_S22'],
 '25-Mar_14_22_44_out.mat': ['GRIN

In [29]:
mouse_file_df = pd.DataFrame(mouse_file_dict)
mouse_file_df.to_csv('/volumes/My_Passport/dlc_analysis/behavcamvideos/cnmfe_file_key.csv')

In [18]:
behav_cam_path = '/volumes/My_Passport/dlc_analysis/behavcamvideos/'
#for mat_file in list(mouse_file_dict.keys()):
    

In [22]:
file = '10-Jan_21_07_47_out.mat'

animal = mouse_file_dict[file][0]
session = mouse_file_dict[file][1]

path_to_tracking = behav_cam_path + animal +'_' + session + '/' + animal + '_' + session + '_dlc_tracking_foranalysis_04142020.csv'
print(path_to_tracking )

/volumes/My_Passport/dlc_analysis/behavcamvideos/GRIN026_H16_M35_S34/GRIN026_H16_M35_S34_dlc_tracking_foranalysis_04142020.csv


In [19]:
session_name=3
dlc_tracking = str(behav_cam_path+mouse_file_dict[list(mouse_file_dict.keys())[session_name]][0]+'_'+mouse_file_dict[list(mouse_file_dict.keys())[session_name]][1]+'/'

'/volumes/My_Passport/dlc_analysis/behavcamvideos/GRIN026_H16_M35_S34'

In [9]:
files_with_errors

['13-Apr_15_30_16_out.mat',
 '26-Mar_22_01_01_out.mat',
 '29-Mar_11_29_08_out.mat',
 '29-Mar_20_07_14_out.mat',
 '30-Mar_11_06_01_out.mat',
 '31-Mar_09_49_26_out.mat']

In [ ]:
results = sio.loadmat('/Volumes/My_Passport/cnmfe_analysis_files/batch_output_files/' + '08-Apr_16_03_01_out.mat')